# Time series comparison between Mpos and FVCOM netCDF
**Author: Jun Sasaki, Coded on January 8, 2024, Updated on January 19, 2024 ([MIT License](https://opensource.org/licenses/MIT))**<br>
- Compare Mpos data and FVCOM output netcdf at a specified siglay. See **dev_mpos_fvcom.ipynb** for development processes.
- **Mpos:** [Monitoring Post](https://www.tbeic.go.jp/MonitoringPost/Top) dataset in Tokyo Bay by MLIT, which should be cleaned and edited using
**create_mpos_nc.ipynb** and **edit_mpos.ipynb**
- Suppose Mpos netCDF and FVCOM output netCDF are prepared.
- FVCOM variables are defined at siglay. The depth at a specified siglay will change with time because of the change in the surface level.
- Interpolate Mpos item values at the specified siglay in FVCOM to precisely compare them.

In [ ]:
from xfvcom import Fvcom, FvcomPlotConfig, FvcomDataArray, FvcomPlotter
from mpos import Mpos
import hvplot.xarray

## Set parameters

In [ ]:
var='temp'
varname='Temperature'
resample='1H'
siglay = 10  # 0: surface
start = "2020-01-01T00:00:00"
end = "2020-12-31T23:00:00"

## Loading Mpos netcdf and creating an mpos instance

In [ ]:
stn = 'kawasaki'  # 'urayasu', 'chibaharo', 'chiba1buoy', 'kawasaki'
year = 2020
dirpath='../Mpos/edited/'
mpos = Mpos(dirpath=dirpath, stn=stn, year=year, nc=True)
mpos.ds

## Loading FVCOM netcdf and creating an `fvcom` instance
```Python
fvcom = xfvcom.Fvcom(dirpath=dirpath, ncfile=ncfile, utm2geo=True, zone=54, north=True, inverse=False)
fvcom.ds  # xarray.Dataset
```

In [ ]:
dirpath = '../../goto2023/output/'
ncfile = 'TokyoBay18_r16_crossed_0001_20s.nc'
fvcom = Fvcom(dirpath=dirpath, ncfile=ncfile, utm2geo=True, zone=54, north=True, inverse=False)
fvcom.ds

## Extracting FVCOM time series depth from the surface `z_dfs` at node
```Python
node = fvcom.nearest_neighbor(lon, lat)  # (lon, lat) -> node
fvcom.ds["z_dfs"][:, siglay, node].sel(time=slice(start, end))  # xarray.DataArray[time, siglay, node]
```

In [ ]:
node_mpos = fvcom.nearest_neighbor(mpos.lon, mpos.lat)
print(f"Node number = {node_mpos}")

In [ ]:
%%script false --no-raise-error
## Editing locally if necessary
siglay = 1  # 0: surface
start = "2020-01-01T00:00:00"
end = "2020-12-31T23:00:00"

In [ ]:
fvcom.ds["z_dfs"][:, siglay, node_mpos].sel(time=slice(start, end))

In [ ]:
fvcom.ds["z_dfs"][:, siglay, node_mpos].sel(time=slice(start, end)).hvplot()

## Resampling by a specified time interval
- The datetime of the FVCOM netcdf may not be hourly or a multiple thereof, but may be slightly off. So it is safe to resampling with interpolation.
- Time index may be different between FVCOM netcdf and Mpos if the time index of Mpos contains that of FVCOM.
```Python
# Resampling by specifying 1 hour interval with linear interpolation
fvcom.ds.resample(time='1H').interpolate('linear')
```
### Resampling by specifying a time origin
- Starting at `'01:00:00'` rather than `'00:00:00'`
```Python
fvcom.ds.resample(time='1H', origin='01:00').interpolate("linear")
```
### Resampling by specifying a `start` and `end` datetime
```Python
fvcom.ds.sel(time=slice(start, end)).resample(time='1H').interpolate('linear')
```
### Downsampling:
- From `1H` To `4H` interval by slicing
```Python
fvcom.ds.sel(time=slice(start, end)).isel(datetime=slice(0, None, 4))
```
- To `4H` interval by resampling with taking mean.
```Python                                     
fvcom.ds.resample(time='4H').mean()
```
### Upsampling
- From '4H` to `1H` interval by intapolation
```Python
fvcom.ds.resample(time='1H').asfreq().interpolate_na(dim='time', method='linear')
# Quick plotting
fvcom.ds.resample(time='1H').asfreq().interpolate_na(dim='time', method='linear').hvplot()
```

In [ ]:
fvcom.ds["z_dfs"][:, siglay, node_mpos].sel(time=slice(start, end)).resample(time='1H').asfreq().interpolate_na(
    dim='time', method='linear').hvplot()

## FVCOM time series depth at (siglay, node) with resapled time interval and range

In [ ]:
z_dfs_ser = fvcom.ds["z_dfs"][:, siglay, node_mpos].sel(time=slice(start, end)).resample(time='1H').asfreq().interpolate_na(
    dim='time', method='linear').to_series()
z_dfs_ser

## Time series var comparison between FVCOM and Mpos
Var names are defined in FVCOM netcdf like below:
```
temp: Temperature, salinity: Salinity
```

In [ ]:
%%script false --no-raise-error
## Editing below if necessary
var='temp'
varname='Temperature'
resample='1H'

### FVCOM time series item at (siglay, node) with resampled time interval and range

In [ ]:
%%script false --no-raise-error
p_fvcom = fvcom.ds[var][:, siglay, node_mpos].sel(time=slice(start, end)).resample(time=resample).asfreq().interpolate_na(
    dim='time', method='linear').hvplot(label='Computed')
p_fvcom

### Using class FvcomDataArray, FvcomPlotConfig, and FvcomPlotter

In [ ]:
fvcom_data = FvcomDataArray(fvcom, var='temp', start=start, end=end, resample=resample)

In [ ]:
fvcom_cfg = FvcomPlotConfig(width=800, height=300)
fvcom_plot = FvcomPlotter(plot_config=fvcom_cfg, fvcom=fvcom_data)

In [ ]:
fvcom_data.da[:, siglay, node_mpos].hvplot()

In [ ]:
p_fvcom = fvcom_plot.ts_hvplot(siglay=siglay, node=node_mpos, title='Kawasaki', ylim=(8, 30), label='Computed')
p_fvcom

### Mpos time series item at the specified depth from the surface (z_dfs)
```Python
# Hide toolbar for html
hvplot.save(plot.options(toolbar=None), 'test.html')
```

In [ ]:
# Time series of observation for a specified item corresponding to FVCOM depth time series
p_mpos = mpos.interp_depth(var, z_dfs_ser, varname).hvplot(label='Measured')
p_mpos

### Exporting html
You may delete a toolbar by setting `toolbar=None` when exporting to html. The html file can be zoomed in to enhance resolution.

In [ ]:
plot = (p_fvcom * p_mpos).options(title='Temperature', legend_position='top_left')
hvplot.save(plot.options(toolbar=None), 'mpos_fvcom.html')
plot

## Close netcdf files

In [ ]:
%%script false --no-raise-error
Mpos.close()
Fvcom.close()